In [1125]:
import pandas as pd
import json
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import URL

In [1126]:
pd.options.display.max_columns = 100

In [1127]:
username = 'root'
password='root'
host='localhost'
database='jmdb'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")


In [1128]:
df = pd.read_csv("emdb.csv",encoding="utf-16", sep = ";")

In [1129]:
df = df.rename(columns = {v : v.strip() for v in df.columns})

In [1130]:
print(df.columns)

Index(['Numéro', 'Titre', 'Titre alternatif', 'Année', 'Pays', 'Genre(s)',
       'Langues', 'Durée', 'Chemin de l'affiche', 'Réalisateur(s)',
       'Scénariste(s)', 'Compositeur(s)', 'Studio', 'Distribution',
       'Bande-annonce', 'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
       'Commentaires', 'Etiquettes', 'Support', 'Taille du fichier',
       'ID TheMovieDb', 'ID IMDb'],
      dtype='object')


# table movie , director et intermediaire

In [1131]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1132]:
# Modifier le chemin de l'affiche pour ne conserver que la dernière partie
df_movie["Chemin de l'affiche"] = df_movie["Chemin de l'affiche"].astype(str).apply(lambda x: x.split('\\')[-1])

# Afficher le DataFrame résultant
print(df_movie.head())  # Affiche les premières lignes du DataFrame

   Numéro                               Titre Titre alternatif Année  Durée  \
0    3504      William Buys a Parrot (vostfr)              NaN  1963      2   
1    3503                 Thot-Fal'N (vostfr)              NaN  1978     14   
2    3502              The Discipline of D.E.              NaN  1982      9   
3    3501                The Cut-Ups (vostfr)              NaN  1966     19   
4    3500  Häxan: Witchcraft Through The Ages              NaN  1968     76   

  Chemin de l'affiche Bande-annonce Synopsis  \
0          003588.jpg           NaN      NaN   
1          003587.jpg           NaN      NaN   
2          003586.jpg           NaN      NaN   
3          003585.jpg           NaN      NaN   
4          003584.jpg           NaN      NaN   

                                            Scénario  \
0  A bizarre silent short film showing author Wil...   
1  This film describes a psychological state 'kin...   
2  The Discipline of D.E."" is a short 16mm film ...   
3  Essential

In [1133]:
list_director_movie = ["Réalisateur(s)", 'Numéro']
df_director_movie = df.copy()[list_director_movie]

In [1134]:
# df_director_movie ["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",")
df_director_movie["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",\s*")

In [1135]:
df_director_movie  = df_director_movie.explode("Réalisateur(s)")

In [1136]:
df_director  = df_director_movie.explode("Réalisateur(s)").drop_duplicates("Réalisateur(s)").reset_index(drop=True).reset_index()
df_director = df_director.rename(columns={"index":"id"}).drop(columns="Numéro")

#.reset_index().rename(columns={"index":"id"})

In [1137]:
df_intermediaire_movie_director = df_director.merge(df_director_movie, on = "Réalisateur(s)").drop(columns="Réalisateur(s)")

In [1138]:
# Remplacer les valeurs NULL de la colonne 'id' par 0
df_director['id'] = df_director['id'].fillna(0)
# Convertir la colonne 'id' en type entier
df_director['id'] = df_director['id'].astype(int)

# Réorganiser les ID si nécessaire pour éviter les conflits
max_id = df_director['id'].max()
if max_id > 0:
    df_director['id'] = df_director['id'] + 1

In [1139]:
display(df_director)

,id,Réalisateur(s)
0,1,Antony Balch
1,2,Stan Brakhage
2,3,Gus Van Sant
3,4,Benjamin Christensen
4,5,Nick Donkin
...,...,...
1733,1734,Richard C. Sarafian
1734,1735,Chris Petit
1735,1736,Brett Ratner
1736,1737,Buster Keaton


In [1140]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3504,William Buys a Parrot (vostfr),NaN,1963,2,003588.jpg,NaN,NaN,A bizarre silent short film showing author Wil...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\William Buy...,Matroska (MKV),NaN,Fichier multimédia,83.0 MB,movie/101810,405470.0
1,3503,Thot-Fal'N (vostfr),NaN,1978,14,003587.jpg,NaN,NaN,This film describes a psychological state 'kin...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\Thot-Fal'N ...,AVI (Microsoft),NaN,Fichier multimédia,172 MB,movie/278859,278772.0
2,3502,The Discipline of D.E.,NaN,1982,9,003586.jpg,NaN,NaN,"The Discipline of D.E."""" is a short 16mm film ...",[Elements]F:\#DIVX\DIVX MOVIEZ\DRAMA\The Disci...,Matroska (MKV),NaN,Fichier multimédia,130 MB,movie/147210,409900.0
3,3501,The Cut-Ups (vostfr),NaN,1966,19,003585.jpg,NaN,NaN,Essentially a dizzying montage of quirky shots...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\The Cut-Ups...,Matroska (MKV),NaN,Fichier multimédia,1007 MB,movie/143497,217346.0
4,3500,Häxan: Witchcraft Through The Ages,NaN,1968,76,003584.jpg,NaN,NaN,A 1968 collaboration between jazz musician Dan...,[Elements]F:\#DIVX\DIVX MOVIEZ\MadMov\Häxan (v...,MPEG-4,NaN,Fichier multimédia,1.11 GB,movie/413708,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,5,le mécano de la général,The General (Titre original),1926,79,000005.jpg,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3500,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,000004.jpg,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,3,Les feux de la rampe,Limelight (Titre original),1952,137,000003.jpg,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3502,2,Les temps modernes,Modern Times (Titre original),1936,87,000002.jpg,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [1141]:
display(df_intermediaire_movie_director)

,id,Numéro
0,0,3504
1,0,3501
2,0,3498
3,0,3496
4,0,3494
...,...,...
3771,1733,20
3772,1734,18
3773,1735,15
3774,1736,5


In [1142]:
# Trouver l'index de la ligne avec le 'Réalisateur(s)' NULL
null_director_index = df_director[df_director['Réalisateur(s)'].isnull()].index

# Si une ligne avec 'Réalisateur(s)' NULL est trouvée
if not null_director_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_director.loc[null_director_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_director['id'].max()
    if max_id > 0:
        df_director.loc[df_director.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_director.head())

# Enregistrer le DataFrame modifié dans la table 'director' de la base de données
df_director.to_sql("director", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_director = df_director.merge(df_director_movie, on="Réalisateur(s)").drop(columns="Réalisateur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_director.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_director.to_sql("movie_director", index=False, con=engine, if_exists="replace")

     id        Réalisateur(s)
0  1739          Antony Balch
1     2         Stan Brakhage
2     3          Gus Van Sant
3     4  Benjamin Christensen
4     5           Nick Donkin
     id  Numéro
0  1739    3504
1  1739    3501
2  1739    3498
3  1739    3496
4  1739    3494


3776

In [1143]:
# df_director.to_sql("director",index=False,con = engine,if_exists = "replace" )

In [1144]:
# df_intermediaire_movie_director.to_sql("movie_director",index=False,con = engine,if_exists = "replace" )

In [1145]:
df_movie.to_sql("movies",index=False,con = engine,if_exists = "replace" )

3504

# table movie, screenwriter et intermediaire

In [1146]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1147]:
list_screenwriter_movie = ["Scénariste(s)", 'Numéro']
df_screenwriter_movie = df.copy()[list_screenwriter_movie]

In [1148]:
df_screenwriter_movie ["Scénariste(s)"] = df_screenwriter_movie["Scénariste(s)"].str.split(",\s*")

In [1149]:
df_screenwriter_movie  = df_screenwriter_movie.explode("Scénariste(s)")

In [1150]:
df_screenwriter  = df_screenwriter_movie.explode("Scénariste(s)").drop_duplicates("Scénariste(s)").reset_index(drop=True).reset_index()
df_screenwriter = df_screenwriter.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1151]:
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on = "Scénariste(s)").drop(columns="Scénariste(s)")

In [1152]:
display(df_screenwriter)

,id,Scénariste(s)
0,0,William S. Burroughs
1,1,NaN
2,2,James Grauerholz
3,3,James Mangold
4,4,Gill Dennis
...,...,...
3800,3800,Ted Tally
3801,3801,David Veloz
3802,3802,Richard Rutowski
3803,3803,Buster Keaton


In [1153]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3504,William Buys a Parrot (vostfr),NaN,1963,2,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A bizarre silent short film showing author Wil...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\William Buy...,Matroska (MKV),NaN,Fichier multimédia,83.0 MB,movie/101810,405470.0
1,3503,Thot-Fal'N (vostfr),NaN,1978,14,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,This film describes a psychological state 'kin...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\Thot-Fal'N ...,AVI (Microsoft),NaN,Fichier multimédia,172 MB,movie/278859,278772.0
2,3502,The Discipline of D.E.,NaN,1982,9,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"The Discipline of D.E."""" is a short 16mm film ...",[Elements]F:\#DIVX\DIVX MOVIEZ\DRAMA\The Disci...,Matroska (MKV),NaN,Fichier multimédia,130 MB,movie/147210,409900.0
3,3501,The Cut-Ups (vostfr),NaN,1966,19,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Essentially a dizzying montage of quirky shots...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\The Cut-Ups...,Matroska (MKV),NaN,Fichier multimédia,1007 MB,movie/143497,217346.0
4,3500,Häxan: Witchcraft Through The Ages,NaN,1968,76,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A 1968 collaboration between jazz musician Dan...,[Elements]F:\#DIVX\DIVX MOVIEZ\MadMov\Häxan (v...,MPEG-4,NaN,Fichier multimédia,1.11 GB,movie/413708,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3500,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3502,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [1154]:
display(df_intermediaire_movie_screenwriter)

,id,Numéro
0,0,3504
1,0,3500
2,0,3499
3,0,3498
4,0,3496
...,...,...
6811,3800,15
6812,3801,14
6813,3802,14
6814,3803,5


In [1155]:
# Trouver l'index de la ligne avec le 'Scénariste(s)' NULL
null_screenwriter_index = df_screenwriter[df_screenwriter['Scénariste(s)'].isnull()].index

# Si une ligne avec 'Scénariste(s)' NULL est trouvée
if not null_screenwriter_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_screenwriter.loc[null_screenwriter_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_screenwriter['id'].max()
    if max_id > 0:
        df_screenwriter.loc[df_screenwriter.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_screenwriter.head())

# Enregistrer le DataFrame modifié dans la table 'screenwriter' de la base de données
df_screenwriter.to_sql("screenwriter", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on="Scénariste(s)").drop(columns="Scénariste(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_screenwriter.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter", index=False, con=engine, if_exists="replace")


     id         Scénariste(s)
0  3805  William S. Burroughs
1     0                   NaN
2     2      James Grauerholz
3     3         James Mangold
4     4           Gill Dennis
     id  Numéro
0  3805    3504
1  3805    3500
2  3805    3499
3  3805    3498
4  3805    3496


6816

In [1156]:
# df_screenwriter.to_sql("screenwriter",index=False,con = engine,if_exists = "replace" )

In [1157]:
# df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter",index=False,con = engine,if_exists = "replace" )

# table movie, music et intermediaire

In [1158]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1159]:
list_music_movie = ["Compositeur(s)", 'Numéro']
df_music_movie = df.copy()[list_music_movie]

In [1160]:
df_music_movie ["Compositeur(s)"] = df_music_movie["Compositeur(s)"].str.split(",\s*")

In [1161]:
df_music_movie  = df_music_movie.explode("Compositeur(s)")

In [1162]:
df_music  = df_music_movie.explode("Compositeur(s)").drop_duplicates("Compositeur(s)").reset_index(drop=True).reset_index()
df_music = df_music.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1163]:
df_intermediaire_movie_music = df_music.merge(df_music_movie, on = "Compositeur(s)").drop(columns="Compositeur(s)")

In [1164]:
display(df_music)

,id,Compositeur(s)
0,0,NaN
1,1,T Bone Burnett
2,2,Reno Isaac
3,3,Benoît de Villeneuve
4,4,Gaspar Claus
...,...,...
1250,1250,Machine Head
1251,1251,Photek
1252,1252,John Frizzell
1253,1253,Leonard Cohen


In [1165]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3504,William Buys a Parrot (vostfr),NaN,1963,2,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A bizarre silent short film showing author Wil...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\William Buy...,Matroska (MKV),NaN,Fichier multimédia,83.0 MB,movie/101810,405470.0
1,3503,Thot-Fal'N (vostfr),NaN,1978,14,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,This film describes a psychological state 'kin...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\Thot-Fal'N ...,AVI (Microsoft),NaN,Fichier multimédia,172 MB,movie/278859,278772.0
2,3502,The Discipline of D.E.,NaN,1982,9,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"The Discipline of D.E."""" is a short 16mm film ...",[Elements]F:\#DIVX\DIVX MOVIEZ\DRAMA\The Disci...,Matroska (MKV),NaN,Fichier multimédia,130 MB,movie/147210,409900.0
3,3501,The Cut-Ups (vostfr),NaN,1966,19,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Essentially a dizzying montage of quirky shots...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\The Cut-Ups...,Matroska (MKV),NaN,Fichier multimédia,1007 MB,movie/143497,217346.0
4,3500,Häxan: Witchcraft Through The Ages,NaN,1968,76,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A 1968 collaboration between jazz musician Dan...,[Elements]F:\#DIVX\DIVX MOVIEZ\MadMov\Häxan (v...,MPEG-4,NaN,Fichier multimédia,1.11 GB,movie/413708,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3500,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3502,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [1166]:
display(df_intermediaire_movie_music)

,id,Numéro
0,0,3504
1,0,3503
2,0,3502
3,0,3501
4,0,3500
...,...,...
3884,1250,46
3885,1251,46
3886,1252,42
3887,1253,14


In [1167]:
# # Trouver l'index de la ligne avec le 'Compositeur(s)' NULL
# null_music_index = df_music[df_music['Compositeur(s)'].isnull()].index

# # Si une ligne avec 'Compositeur(s)' NULL est trouvée
# if not null_music_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_music.loc[null_music_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_music['id'].max()
#     if max_id > 0:
#         df_music.loc[df_music.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_music.head())

# # Enregistrer le DataFrame modifié dans la table 'music' de la base de données
# df_music.to_sql("music", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_music = df_music.merge(df_music_movie, on="Compositeur(s)").drop(columns="Compositeur(s)")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_music.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_music.to_sql("movie_music", index=False, con=engine, if_exists="replace")


In [1168]:
df_music.to_sql("music",index=False,con = engine,if_exists = "replace" )

1255

In [1169]:
df_intermediaire_movie_music.to_sql("movie_music",index=False,con = engine,if_exists = "replace" )

3889

# table movie, studio et intermediaire

In [1170]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1171]:
list_studio_movie = ["Studio", 'Numéro']
df_studio_movie = df.copy()[list_studio_movie]

In [1172]:
df_studio_movie ["Studio"] = df_studio_movie["Studio"].str.split(",\s*")

In [1173]:
df_studio_movie  = df_studio_movie.explode("Studio")

In [1174]:
df_studio  = df_studio_movie.explode("Studio").drop_duplicates("Studio").reset_index(drop=True).reset_index()
df_studio = df_studio.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1175]:
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on = "Studio").drop(columns="Studio")

In [1176]:
display(df_studio)

,id,Studio
0,0,NaN
1,1,VH1
2,2,Antony Balch Films
3,3,Pinball London
4,4,Filmakademie Baden-Württemberg
...,...,...
918,918,A-Film Distribution
919,919,TFC
920,920,Dybex
921,921,British Empire Films Australia


In [1177]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3504,William Buys a Parrot (vostfr),NaN,1963,2,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A bizarre silent short film showing author Wil...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\William Buy...,Matroska (MKV),NaN,Fichier multimédia,83.0 MB,movie/101810,405470.0
1,3503,Thot-Fal'N (vostfr),NaN,1978,14,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,This film describes a psychological state 'kin...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\Thot-Fal'N ...,AVI (Microsoft),NaN,Fichier multimédia,172 MB,movie/278859,278772.0
2,3502,The Discipline of D.E.,NaN,1982,9,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"The Discipline of D.E."""" is a short 16mm film ...",[Elements]F:\#DIVX\DIVX MOVIEZ\DRAMA\The Disci...,Matroska (MKV),NaN,Fichier multimédia,130 MB,movie/147210,409900.0
3,3501,The Cut-Ups (vostfr),NaN,1966,19,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Essentially a dizzying montage of quirky shots...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\The Cut-Ups...,Matroska (MKV),NaN,Fichier multimédia,1007 MB,movie/143497,217346.0
4,3500,Häxan: Witchcraft Through The Ages,NaN,1968,76,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A 1968 collaboration between jazz musician Dan...,[Elements]F:\#DIVX\DIVX MOVIEZ\MadMov\Häxan (v...,MPEG-4,NaN,Fichier multimédia,1.11 GB,movie/413708,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3500,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3502,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [1178]:
display(df_intermediaire_movie_studio)

,id,Numéro
0,0,3504
1,0,3503
2,0,3502
3,0,3501
4,0,3500
...,...,...
3501,918,172
3502,919,171
3503,920,169
3504,921,108


In [1179]:
# # Trouver l'index de la ligne avec le 'Studio' NULL
# null_studio_index = df_studio[df_studio['Studio'].isnull()].index

# # Si une ligne avec 'Studio' NULL est trouvée
# if not null_studio_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_studio.loc[null_studio_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_studio['id'].max()
#     if max_id > 0:
#         df_studio.loc[df_studio.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_studio.head())

# # Enregistrer le DataFrame modifié dans la table 'studio' de la base de données
# df_studio.to_sql("studio", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on="Studio").drop(columns="Studio")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_studio.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_studio.to_sql("movie_studio", index=False, con=engine, if_exists="replace")


In [1180]:
df_studio.to_sql("studio",index=False,con = engine,if_exists = "replace" )

923

In [1181]:
df_intermediaire_movie_studio.to_sql("movie_studio",index=False,con = engine,if_exists = "replace" )

3506

# table movie, casting & intermediaire

In [1182]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1183]:
list_casting_movie = ["Distribution", 'Numéro']
df_casting_movie = df.copy()[list_casting_movie]

In [1184]:
df_casting_movie ["Distribution"] = df_casting_movie["Distribution"].str.split(",\s*")

In [1185]:
df_casting_movie  = df_casting_movie.explode("Distribution")

In [1186]:
df_casting  = df_casting_movie.explode("Distribution").drop_duplicates("Distribution").reset_index(drop=True).reset_index()
df_casting = df_casting.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1187]:
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on = "Distribution").drop(columns="Distribution")

In [1188]:
display(df_casting)

,id,Distribution
0,0,William S. Burroughs
1,1,Jane Brakhage
2,2,Tom Bartek
3,3,Gloria Bartek
4,4,Allen Ginsberg
...,...,...
9631,9631,Paulette Goddard
9632,9632,Henry Bergman
9633,9633,Tiny Sandford
9634,9634,Jackie Coogan


In [1189]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3504,William Buys a Parrot (vostfr),NaN,1963,2,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A bizarre silent short film showing author Wil...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\William Buy...,Matroska (MKV),NaN,Fichier multimédia,83.0 MB,movie/101810,405470.0
1,3503,Thot-Fal'N (vostfr),NaN,1978,14,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,This film describes a psychological state 'kin...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\Thot-Fal'N ...,AVI (Microsoft),NaN,Fichier multimédia,172 MB,movie/278859,278772.0
2,3502,The Discipline of D.E.,NaN,1982,9,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"The Discipline of D.E."""" is a short 16mm film ...",[Elements]F:\#DIVX\DIVX MOVIEZ\DRAMA\The Disci...,Matroska (MKV),NaN,Fichier multimédia,130 MB,movie/147210,409900.0
3,3501,The Cut-Ups (vostfr),NaN,1966,19,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Essentially a dizzying montage of quirky shots...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\The Cut-Ups...,Matroska (MKV),NaN,Fichier multimédia,1007 MB,movie/143497,217346.0
4,3500,Häxan: Witchcraft Through The Ages,NaN,1968,76,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A 1968 collaboration between jazz musician Dan...,[Elements]F:\#DIVX\DIVX MOVIEZ\MadMov\Häxan (v...,MPEG-4,NaN,Fichier multimédia,1.11 GB,movie/413708,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3500,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3502,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [1190]:
display(df_intermediaire_movie_casting)

,id,Numéro
0,0,3504
1,0,3503
2,0,3501
3,0,3500
4,0,3499
...,...,...
16614,9631,2
16615,9632,2
16616,9633,2
16617,9634,1


In [1191]:
# Trouver l'index de la ligne avec la 'Distribution' NULL
null_casting_index = df_casting[df_casting['Distribution'].isnull()].index

# Si une ligne avec 'Distribution' NULL est trouvée
if not null_casting_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_casting.loc[null_casting_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_casting['id'].max()
    if max_id > 0:
        df_casting.loc[df_casting.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_casting.head())

# Enregistrer le DataFrame modifié dans la table 'casting' de la base de données
df_casting.to_sql("casting", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on="Distribution").drop(columns="Distribution")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_casting.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_casting.to_sql("movie_casting", index=False, con=engine, if_exists="replace")


     id          Distribution
0  9636  William S. Burroughs
1     1         Jane Brakhage
2     2            Tom Bartek
3     3         Gloria Bartek
4     4        Allen Ginsberg
     id  Numéro
0  9636    3504
1  9636    3503
2  9636    3501
3  9636    3500
4  9636    3499


16619

In [1192]:
# df_casting.to_sql("casting",index=False,con = engine,if_exists = "replace" )

In [1193]:
# df_intermediaire_movie_casting.to_sql("movie_casting",index=False,con = engine,if_exists = "replace" )

# table movie, country & intermediaire

In [1194]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1195]:
list_country_movie = ["Pays", 'Numéro']
df_country_movie = df.copy()[list_country_movie]

In [1196]:
df_country_movie ["Pays"] = df_country_movie["Pays"].str.split(",\s*")

In [1197]:
df_country_movie  = df_country_movie.explode("Pays")

In [1198]:
df_country  = df_country_movie.explode("Pays").drop_duplicates("Pays").reset_index(drop=True).reset_index()
df_country = df_country.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1199]:
df_intermediaire_movie_country = df_country.merge(df_country_movie, on = "Pays").drop(columns="Pays")

In [1200]:
display(df_country)

,id,Pays
0,0,Etats-Unis
1,1,Royaume-Uni
2,2,Allemagne
3,3,France
4,4,Japon
5,5,Corée du Sud
6,6,PS
7,7,Italie
8,8,Estonie
9,9,XC


In [1201]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3504,William Buys a Parrot (vostfr),NaN,1963,2,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A bizarre silent short film showing author Wil...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\William Buy...,Matroska (MKV),NaN,Fichier multimédia,83.0 MB,movie/101810,405470.0
1,3503,Thot-Fal'N (vostfr),NaN,1978,14,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,This film describes a psychological state 'kin...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\Thot-Fal'N ...,AVI (Microsoft),NaN,Fichier multimédia,172 MB,movie/278859,278772.0
2,3502,The Discipline of D.E.,NaN,1982,9,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"The Discipline of D.E."""" is a short 16mm film ...",[Elements]F:\#DIVX\DIVX MOVIEZ\DRAMA\The Disci...,Matroska (MKV),NaN,Fichier multimédia,130 MB,movie/147210,409900.0
3,3501,The Cut-Ups (vostfr),NaN,1966,19,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Essentially a dizzying montage of quirky shots...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\The Cut-Ups...,Matroska (MKV),NaN,Fichier multimédia,1007 MB,movie/143497,217346.0
4,3500,Häxan: Witchcraft Through The Ages,NaN,1968,76,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A 1968 collaboration between jazz musician Dan...,[Elements]F:\#DIVX\DIVX MOVIEZ\MadMov\Häxan (v...,MPEG-4,NaN,Fichier multimédia,1.11 GB,movie/413708,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3500,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3502,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [1202]:
display(df_intermediaire_movie_country)

,id,Numéro
0,0,3504
1,0,3503
2,0,3502
3,0,3500
4,0,3499
...,...,...
3499,58,426
3500,58,231
3501,59,206
3502,59,201


In [1203]:
# # Trouver l'index de la ligne avec le 'Pays' NULL
# null_country_index = df_country[df_country['Pays'].isnull()].index

# # Si une ligne avec 'Pays' NULL est trouvée
# if not null_country_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_country.loc[null_country_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_country['id'].max()
#     if max_id > 0:
#         df_country.loc[df_country.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_country.head())

# # Enregistrer le DataFrame modifié dans la table 'country' de la base de données
# df_country.to_sql("country", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_country = df_country.merge(df_country_movie, on="Pays").drop(columns="Pays")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_country.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_country.to_sql("movie_country", index=False, con=engine, if_exists="replace")

In [1204]:
df_country.to_sql("country",index=False,con = engine,if_exists = "replace" )

60

In [1205]:
df_intermediaire_movie_country.to_sql("movie_country",index=False,con = engine,if_exists = "replace" )

3504

# table movie, genre & intermediaire

In [1206]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1207]:
list_genre_movie = ["Genre(s)", 'Numéro']
df_genre_movie = df.copy()[list_genre_movie]

In [1208]:
df_genre_movie ["Genre(s)"] = df_genre_movie["Genre(s)"].str.split(",\s*")

In [1209]:
df_genre_movie  = df_genre_movie.explode("Genre(s)")

In [1210]:
df_genre  = df_genre_movie.explode("Genre(s)").drop_duplicates("Genre(s)").reset_index(drop=True).reset_index()
df_genre = df_genre.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1211]:
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on = "Genre(s)").drop(columns="Genre(s)")

In [1212]:
display(df_genre)

,id,Genre(s)
0,0,Documentaire
1,1,Drame
2,2,Comédie
3,3,Court-métrage
4,4,Histoire
5,5,Horreur
6,6,Animation
7,7,Fantastique
8,8,Biopic
9,9,Musical


In [1213]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3504,William Buys a Parrot (vostfr),NaN,1963,2,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A bizarre silent short film showing author Wil...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\William Buy...,Matroska (MKV),NaN,Fichier multimédia,83.0 MB,movie/101810,405470.0
1,3503,Thot-Fal'N (vostfr),NaN,1978,14,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,This film describes a psychological state 'kin...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\Thot-Fal'N ...,AVI (Microsoft),NaN,Fichier multimédia,172 MB,movie/278859,278772.0
2,3502,The Discipline of D.E.,NaN,1982,9,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"The Discipline of D.E."""" is a short 16mm film ...",[Elements]F:\#DIVX\DIVX MOVIEZ\DRAMA\The Disci...,Matroska (MKV),NaN,Fichier multimédia,130 MB,movie/147210,409900.0
3,3501,The Cut-Ups (vostfr),NaN,1966,19,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Essentially a dizzying montage of quirky shots...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\The Cut-Ups...,Matroska (MKV),NaN,Fichier multimédia,1007 MB,movie/143497,217346.0
4,3500,Häxan: Witchcraft Through The Ages,NaN,1968,76,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A 1968 collaboration between jazz musician Dan...,[Elements]F:\#DIVX\DIVX MOVIEZ\MadMov\Häxan (v...,MPEG-4,NaN,Fichier multimédia,1.11 GB,movie/413708,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3500,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3502,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [1214]:
display(df_intermediaire_movie_genre)

,id,Numéro
0,0,3504
1,0,3503
2,0,3501
3,0,3498
4,0,3496
...,...,...
7868,38,800
7869,38,690
7870,38,182
7871,38,181


In [1215]:
# Trouver l'index de la ligne avec le 'Genre(s)' NULL
null_genre_index = df_genre[df_genre['Genre(s)'].isnull()].index

# Si une ligne avec 'Genre(s)' NULL est trouvée
if not null_genre_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_genre.loc[null_genre_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_genre['id'].max()
    if max_id > 0:
        df_genre.loc[df_genre.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_genre.head())

# Enregistrer le DataFrame modifié dans la table 'genre' de la base de données
df_genre.to_sql("genre", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on="Genre(s)").drop(columns="Genre(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_genre.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_genre.to_sql("movie_genre", index=False, con=engine, if_exists="replace")


   id       Genre(s)
0  39   Documentaire
1   1          Drame
2   2        Comédie
3   3  Court-métrage
4   4       Histoire
   id  Numéro
0  39    3504
1  39    3503
2  39    3501
3  39    3498
4  39    3496


7873

In [1216]:
df_genre.to_sql("genre",index=False,con = engine,if_exists = "replace" )

39

In [1217]:
df_intermediaire_movie_genre.to_sql("movie_genre",index=False,con = engine,if_exists = "replace" )

7873

# table movie, language & intermediaire

In [1218]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1219]:
list_language_movie = ["Langues", 'Numéro']
df_language_movie = df.copy()[list_language_movie]

In [1220]:
df_language_movie ["Langues"] = df_language_movie["Langues"].str.split(",\s*")

In [1221]:
df_language_movie  = df_language_movie.explode("Langues")

In [1222]:
df_language  = df_language_movie.explode("Langues").drop_duplicates("Langues").reset_index(drop=True).reset_index()
df_language = df_language.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1223]:
df_intermediaire_movie_language = df_language.merge(df_language_movie, on = "Langues").drop(columns="Langues")

In [1224]:
display(df_language)

,id,Langues
0,0,NaN
1,1,Anglais
2,2,Russe
3,3,Français
4,4,Japonais
5,5,Coréen
6,6,Italien
7,7,Estonien
8,8,Espagnol
9,9,Allemand


In [1225]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3504,William Buys a Parrot (vostfr),NaN,1963,2,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A bizarre silent short film showing author Wil...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\William Buy...,Matroska (MKV),NaN,Fichier multimédia,83.0 MB,movie/101810,405470.0
1,3503,Thot-Fal'N (vostfr),NaN,1978,14,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,This film describes a psychological state 'kin...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\Thot-Fal'N ...,AVI (Microsoft),NaN,Fichier multimédia,172 MB,movie/278859,278772.0
2,3502,The Discipline of D.E.,NaN,1982,9,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"The Discipline of D.E."""" is a short 16mm film ...",[Elements]F:\#DIVX\DIVX MOVIEZ\DRAMA\The Disci...,Matroska (MKV),NaN,Fichier multimédia,130 MB,movie/147210,409900.0
3,3501,The Cut-Ups (vostfr),NaN,1966,19,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Essentially a dizzying montage of quirky shots...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\The Cut-Ups...,Matroska (MKV),NaN,Fichier multimédia,1007 MB,movie/143497,217346.0
4,3500,Häxan: Witchcraft Through The Ages,NaN,1968,76,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A 1968 collaboration between jazz musician Dan...,[Elements]F:\#DIVX\DIVX MOVIEZ\MadMov\Häxan (v...,MPEG-4,NaN,Fichier multimédia,1.11 GB,movie/413708,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3500,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3502,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [1226]:
display(df_intermediaire_movie_language)

,id,Numéro
0,0,3504
1,0,3503
2,0,3497
3,0,3477
4,0,3476
...,...,...
4937,42,1060
4938,43,1053
4939,43,930
4940,43,926


In [1227]:
# # Trouver l'index de la ligne avec les 'Langues' NULL
# null_language_index = df_language[df_language['Langues'].isnull()].index

# # Si une ligne avec 'Langues' NULL est trouvée
# if not null_language_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_language.loc[null_language_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_language['id'].max()
#     if max_id > 0:
#         df_language.loc[df_language.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_language.head())

# # Enregistrer le DataFrame modifié dans la table 'language' de la base de données
# df_language.to_sql("language", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_language = df_language.merge(df_language_movie, on="Langues").drop(columns="Langues")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_language.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_language.to_sql("movie_language", index=False, con=engine, if_exists="replace")


In [1228]:
df_language.to_sql("language",index=False,con = engine,if_exists = "replace" )

45

In [1229]:
df_intermediaire_movie_language.to_sql("movie_language",index=False,con = engine,if_exists = "replace" )

4942

# table, movie, tag & intermediaire

In [1230]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1231]:
list_tag_movie = ["Etiquettes", 'Numéro']
df_tag_movie = df.copy()[list_tag_movie]

In [1232]:
df_tag_movie ["Etiquettes"] = df_tag_movie["Etiquettes"].str.split(",\s*")

In [1233]:
df_tag_movie  = df_tag_movie.explode("Etiquettes")

In [1234]:
df_tag = df_tag_movie.explode("Etiquettes").drop_duplicates("Etiquettes").reset_index(drop=True).reset_index()
df_tag = df_tag.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1235]:
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on = "Etiquettes").drop(columns="Etiquettes")

In [1236]:
display(df_tag)

,id,Etiquettes
0,0,Burroughs
1,1,Diable / Démons
2,2,Sorcière
3,3,Experimental
4,4,NaN
...,...,...
200,200,Momie
201,201,viking
202,202,Réalisme soviétique
203,203,Guy Debord


In [1237]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3504,William Buys a Parrot (vostfr),NaN,1963,2,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A bizarre silent short film showing author Wil...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\William Buy...,Matroska (MKV),NaN,Fichier multimédia,83.0 MB,movie/101810,405470.0
1,3503,Thot-Fal'N (vostfr),NaN,1978,14,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,This film describes a psychological state 'kin...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\Thot-Fal'N ...,AVI (Microsoft),NaN,Fichier multimédia,172 MB,movie/278859,278772.0
2,3502,The Discipline of D.E.,NaN,1982,9,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"The Discipline of D.E."""" is a short 16mm film ...",[Elements]F:\#DIVX\DIVX MOVIEZ\DRAMA\The Disci...,Matroska (MKV),NaN,Fichier multimédia,130 MB,movie/147210,409900.0
3,3501,The Cut-Ups (vostfr),NaN,1966,19,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Essentially a dizzying montage of quirky shots...,[Elements]F:\#DIVX\DIVX MOVIEZ\DOC\The Cut-Ups...,Matroska (MKV),NaN,Fichier multimédia,1007 MB,movie/143497,217346.0
4,3500,Häxan: Witchcraft Through The Ages,NaN,1968,76,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,A 1968 collaboration between jazz musician Dan...,[Elements]F:\#DIVX\DIVX MOVIEZ\MadMov\Häxan (v...,MPEG-4,NaN,Fichier multimédia,1.11 GB,movie/413708,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3500,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3502,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [1238]:
display(df_intermediaire_movie_tag)

,id,Numéro
0,0,3504
1,0,3503
2,0,3502
3,0,3501
4,0,3500
...,...,...
4448,203,74
4449,203,73
4450,204,17
4451,204,16


In [1239]:
# Trouver l'index de la ligne avec les 'Etiquettes' NULL
null_tag_index = df_tag[df_tag['Etiquettes'].isnull()].index

# Si une ligne avec 'Etiquettes' NULL est trouvée
if not null_tag_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_tag.loc[null_tag_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_tag['id'].max()
    if max_id > 0:
        df_tag.loc[df_tag.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_tag.head())

# Enregistrer le DataFrame modifié dans la table 'tag' de la base de données
df_tag.to_sql("tag", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on="Etiquettes").drop(columns="Etiquettes")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_tag.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_tag.to_sql("movie_tag", index=False, con=engine, if_exists="replace")


    id       Etiquettes
0  205        Burroughs
1    1  Diable / Démons
2    2         Sorcière
3    3     Experimental
4    0              NaN
    id  Numéro
0  205    3504
1  205    3503
2  205    3502
3  205    3501
4  205    3500


4453

In [1240]:
# df_tag.to_sql("tag",index=False,con = engine,if_exists = "replace" )

In [1241]:
# df_intermediaire_movie_tag.to_sql("movie_tag",index=False,con = engine,if_exists = "replace" )